---

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm
from ast import literal_eval

# Initial data collecting

To get data from all the starwars characters from the official Starwars Databank, we first need to know all the possible URLs to get the data from. This is done by getting the list of all the characters from the 'starwars-databank-server.vercel.app' API, which is then appended to the base URL:

In [ ]:
URL_CHARACTERS = "https://starwars-databank-server.vercel.app/api/v1/characters"

def getResponse(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError(f"Failed to get data. Status code: {response.status_code}")
    data = response.json()
    return data

def cleanString(var):
    return str(var).replace("\"", "\'")

def getNextUrl(data):
    BASE_URL = "https://starwars-databank-server.vercel.app/"
    try: return (BASE_URL + data["info"]["next"])
    except: return None

In [ ]:
character_df_old = None
temp_url = URL_CHARACTERS
character_name_list = []

for _ in tqdm(range(100)):
    data = getResponse(temp_url)
    character_name_list += [cleanString(character["name"]) for character in data["data"]]
    
    # Since the getResponse function doesn't return all characters
    # we need to loop through the different API "pages" with getNextUrl
    temp_url = getNextUrl(data)
    if temp_url is None:
        print("Done!")
        break

url_name_list = ["-".join(name.lower().replace("\'", "").split()) for name in character_name_list]

 96%|█████████▌| 96/100 [00:54<00:02,  1.75it/s]

Done!


Now that we have a list of character names we can then start getting the character data we actually want from the official Star Wars databank website. So we start appending and checking the different URLs, from the resulting HTML pages we can then extract the desired data:

In [ ]:
BASE_URL = "https://www.starwars.com/databank/"
URL_ADDITIONS = url_name_list

errors = []
number_of_errors = 0

def add_one_error():
    global number_of_errors
    number_of_errors += 1

def get_page(url):
    try: response = requests.get(url)
    except: errors.append(url); add_one_error(); return None
    return BeautifulSoup(response.text, "html.parser")

def check_if_error_404(soup):
    try: soup.find_all("p", class_="desc")[0].text; return False
    except: return True

def get_properties(soup):
    stats = soup.find_all("section", class_="module stats span-full-screen content-span-full-screen secondary-theme dark")
    categories = stats[0].find_all("div", class_="category")
    properties = {}
    for category in categories:
        heading = category.find_all("div", class_="heading")[0].text
        property_names = [p.text for p in category.find_all("div", class_="property-name")]
        properties[heading] = property_names
    return properties

def get_description(soup):
    discription = soup.find_all("p", class_="desc")[0].text
    discription = discription.replace("\n", "")
    return discription

def get_history(soup):
    all_referals = []
    all_text = soup.find_all("div", class_="rich-text-output")
    if len(all_text) == 0:
        return None, None
    minor_text_samples = all_text[0].find_all("p")
    all_history_text = ""
    for sample in minor_text_samples:
        referals = sample.find_all("a")
        for referal in referals:
            all_referals.append(referal.text.strip())
        all_history_text += " " + sample.text
    return all_history_text, all_referals

def get_descriptions(url):
    soup = get_page(url)
    if soup is None or check_if_error_404(soup):
        errors.append(url)
        add_one_error()
        return None
    description = get_description(soup)
    history, referal = get_history(soup)
    properties = get_properties(soup)
    return description, referal, properties, history

#### Getting all the correct URL's before proceeding

We noticed that some of the automade URL's did not work properly. We then saved these to an error.csv file:

In [ ]:
def check_if_working_url(url):
    soup = get_page(url)
    return not (soup is None or check_if_error_404(soup))

In [ ]:
all_urls = []
errors = []

for i in tqdm(range(len(character_name_list))):
    name = character_name_list[i]
    url_addition = URL_ADDITIONS[i]
    full_url = BASE_URL + url_addition
    if check_if_working_url(full_url):
        all_urls.append((name, full_url))
    else:
        errors.append((name, full_url))

100%|██████████| 964/964 [11:32<00:00,  1.39it/s]


### Concat cleaned errors and all_urls

The error.csv was only 43 lines long, so we decided to manully search for the person on the star-wars databank, and then copy the correct html back into the error.csv, as we found this to be most efficient and accurate way of correcting the mistakes:

In [ ]:
potentially_cleaned_urls = []

cleaned_errors = [lst.split("\n") for lst in open("cleaned_errors.csv", "r").readlines()]
for cleaned_error in cleaned_errors:
    potentially_cleaned_urls.append(cleaned_error[0])

In [ ]:
errors = []

for i in tqdm(range(len( potentially_cleaned_urls))):
    name = literal_eval(potentially_cleaned_urls[i])[0]
    full_url = literal_eval(potentially_cleaned_urls[i])[1]
    if check_if_working_url(full_url):
        all_urls.append((name, full_url))
    else:
        errors.append((name, full_url))

100%|██████████| 68/68 [00:41<00:00,  1.63it/s]


In [ ]:
with open("errors.csv", "w") as f:
    for line in errors:
        f.write(str(line) + "\n")

We then had an errors.csv with no lines left, which is exactly what we wanted.

### Making the dataframe

In [ ]:
# creating dataframe objects:
new_names = []
errors = []
descriptions = []
referrals = []
properties = []
histories = []
number_of_errors = 0

for i in tqdm(range(len(all_urls))):
    name = all_urls[i][0]
    full_url = all_urls[i][1]
    descriptions_tuple = get_descriptions(full_url)

    if descriptions_tuple is None:
        continue 
    
    description, referral, property, history = descriptions_tuple
    new_names.append(name)
    descriptions.append(description)
    referrals.append(referral)
    properties.append(property)
    histories.append(history)

    all_urls.append((name, full_url))  

print("Number of errors: ", number_of_errors)
print(*errors, sep="\n") 

cleaned_descriptions = [description.replace("\n", "").replace("\r", "") for description in descriptions]
df = pd.DataFrame({"name": new_names, "description": cleaned_descriptions, "referrals": referrals, "properties": properties, "history": history})
df.to_csv("descriptions.csv", sep=";")

100%|██████████| 958/958 [10:44<00:00,  1.49it/s]

Number of errors:  0



In [ ]:
print("Number of errors: ", number_of_errors)
with open("errors.csv", "w") as f:
    for line in errors:
        f.write(line + "\n")

Number of errors:  0


We now have a dataset with a lot of information about each character:

In [6]:
import pandas as pd
df = pd.read_csv("descriptions.csv", sep=";")
df

,Unnamed: 0,name,description,referrals,properties,history
0,0,'Hyper' Rod,"In his specially-modified Seven Deuce Blaster,...",NaN,{'Appearances': ['Star Wars: The Bad Batch']},NaN
1,1,'Quick-Draw' Quasar,"An amalgamation of droid parts, riot racer ""Qu...",NaN,{'Appearances': ['Star Wars: The Bad Batch']},NaN
2,2,'Steel Claw' Kane,"At the Safa Toma riot races, ""Steel Claw"" Kane...",NaN,"{'Appearances': ['Star Wars: The Bad Batch'], ...",NaN
3,3,'The Trickster' Flash Raktor,Perhaps riot racer Flash Raktor gets his nickn...,NaN,"{'Appearances': ['Star Wars: The Bad Batch'], ...",NaN
4,4,2BR-NTB,"Loyal to Norath Kev, the droid 2BR-NTB has a d...",NaN,{'Appearances': ['Star Wars Resistance']},NaN
...,...,...,...,...,...,...
953,953,Venisa Doza,"A fearless freedom fighter, Venisa Doza once f...","['the Resistance', 'Colossus', 'Jade Squadron']","{'Appearances': ['Star Wars Resistance'], 'Gen...",NaN
954,954,Wicket W. Warrick,Wicket W. Warrick was the brave young Ewok who...,"['Empire', 'Endor', 'Ewok', 'Bright Tree Villa...",{'Appearances': ['Star Wars: Return of the Jed...,NaN
955,955,Willie (W1-LE),W1-LE was Lando Calrissian’s personal attendan...,NaN,"{'Appearances': ['Star Wars Rebels'], 'droid':...",NaN
956,956,Xi'an,"An athletic and agile Twi'lek, Xi'an is the bl...",NaN,"{'Appearances': ['The Mandalorian'], 'Gender':...",NaN


---